In [ ]:
from __future__ import division, print_function

import GPflow
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
from scipy import linalg
%matplotlib inline
import tensorflow as tf
from scipy.optimize import minimize

try:
    from tqdm import tqdm
except ImportError:
    tqdm = lambda x: x

import safe_learning
import plotting

try:
    session.close()
except NameError:
    pass

session = tf.InteractiveSession()
session.run(tf.global_variables_initializer())

### Goal:

Optimize over the policy such that the safe set does not shrink

In [ ]:
n = 2
m = 1

# 'Wrong' model parameters
mass = 0.1
friction = 0.
length = 0.5
gravity = 9.81
inertia = mass * length ** 2

# True model parameters
true_mass = 0.15
true_friction = 0.05
true_length = length
true_inertia = true_mass * true_length ** 2

# Input saturation
x_max = np.deg2rad(30)
u_max = gravity * true_mass * true_length * np.sin(x_max)

norm_state = np.array([x_max, np.sqrt(gravity / length)])
norm_action = np.array([u_max])


In [ ]:
from scipy import signal

class InvertedPendulum(safe_learning.DeterministicFunction):
    """Inverted Pendulum.
    
    Parameters
    ----------
    mass : float
    length : float
    friction : float, optional
    dt : float, optional
        The sampling time.
    normalization : tuple, optional
        A tuple (Tx, Tu) of arrays used to normalize the state and actions. It
        is so that diag(Tx) *x_norm = x and diag(Tu) * u_norm = u.
    """
    def __init__(self, mass, length, friction=0, dt=1/80, normalization=None):
        super(InvertedPendulum, self).__init__()
        self.mass = mass
        self.length = length
        self.gravity = 9.81
        self.friction = friction
        self.dt = dt
        
        self.normalization = normalization
        if normalization is not None:
            self.normalization = [arr.astype(safe_learning.config.np_dtype) for arr in normalization]
            self.inv_norm = [arr ** -1 for arr in self.normalization]
            
    @property
    def inertia(self):
        """The inertia of the pendulum"""
        return self.mass * self.length ** 2
    
    def normalize(self, state, action):
        """Normalize states and actions."""
        if self.normalization is None:
            return state, action
        
        Tx_inv, Tu_inv = map(np.diag, self.inv_norm)
        state = tf.matmul(state, Tx_inv)
        
        if action is not None:
            action = tf.matmul(action, Tu)
            
        return state, action
    
    def denormalize(self, state, action):
        """Denormalize states and actions"""
        if self.normalization is None:
            return state, action
        
        Tx, Tu = map(np.diag, self.normalization)
        
        state = tf.matmul(state, Tx)
        if action is not None:
            action = tf.matmul(action, Tu)
        
        return state, action
    
    def linearize(self):
        """Return the linearized system.
        
        Returns
        -------
        a : ndarray
            The state matrix.
        b : ndarray
            The action matrix.
        """
        gravity = self.gravity
        length = self.length
        friction = self.friction
        inertia = self.inertia
        
        A = np.array([[0, 1],
                      [gravity / length, -friction / inertia]],
                    dtype=safe_learning.config.np_dtype)
        
        B = np.array([[0],
                      [1 / inertia]],
                    dtype=safe_learning.config.np_dtype)
        
        if self.normalization is not None:
            Tx, Tu = map(np.diag, self.normalization)
            Tx_inv, Tu_inv = map(np.diag, self.inv_norm)
            
            A = np.linalg.multi_dot((Tx_inv, A, Tx))
            B = np.linalg.multi_dot((Tx_inv, B, Tu))
            
        sys = signal.StateSpace(A, B, np.eye(2), np.zeros((2, 1)))
        sysd = sys.to_discrete(self.dt)
        return sysd.A, sysd.B
    
    @safe_learning.utilities.concatenate_inputs(start=1)
    def evaluate(self, state_action):
        """Evaluate the dynamics"""
        # Denormalize
        state, action = tf.split(state_action, [2,  1], axis=1)
        state, action = self.denormalize(state, action)
        
        n_inner = 10
        dt = self.dt / n_inner
        for i in range(n_inner):
            state_derivative = self.ode(state, action)
            state = state + dt * state_derivative
            
        return self.normalize(state, None)[0]
        
    def ode(self, state, action):
        """Compute the state time-derivative.
    
        Parameters
        ----------
        states: ndarray or Tensor
            Unnormalized states.
        actions: ndarray or Tensor
            Unnormalized actions.
            
        Returns
        -------
        x_dot: Tensor
            The normalized derivative of the dynamics
        """

        # Physical dynamics
        gravity = self.gravity
        length = self.length
        friction = self.friction
        inertia = self.inertia
        
        angle, angular_velocity = tf.split(state, 2, axis=1)
        
        x_ddot = gravity / length * tf.sin(angle) + action / inertia
        
        if friction > 0:
            x_ddot -= friction / inertia * angular_velocity
        
        state_derivative = tf.concat((angular_velocity, x_ddot), axis=1)

        # Normalize
        return state_derivative #self.normalize(state_derivative, None)[0]
    
    


In [ ]:
true_dynamics = InvertedPendulum(mass=true_mass, length=true_length, friction=true_friction,
                                normalization=(norm_state, norm_action))

wrong_pendulum = InvertedPendulum(mass=mass, length=length, friction=friction,
                                  normalization=(norm_state, norm_action))

# LQR cost matrices
q = 1 * np.diag([1., 2.])
r = 1.2 * np.array([[1]], dtype=safe_learning.config.np_dtype)

In [ ]:
# x_min, x_max, discretization\
state_limits = np.array([[-1, 1], [-1., 1.]])
action_limits = np.array([[-1, 1]])
num_states = [2001, 2001]
num_actions = [101]

discretization = safe_learning.GridWorld(np.vstack((state_limits, action_limits)),
                                         num_states + num_actions)

safety_disc = safe_learning.GridWorld(state_limits, num_states)
action_disc = safe_learning.GridWorld(action_limits, num_actions)
policy_disc = safe_learning.GridWorld(state_limits, [55, 55])

# Discretization constant
tau = np.min(safety_disc.unit_maxes)

print('Grid size: {0}'.format(safety_disc.nindex))

### Define GP dynamics model

In [ ]:
A, B = wrong_pendulum.linearize()
lipschitz_dynamics = 1

noise_var = 0.001 ** 2

m_true = np.hstack((true_dynamics.linearize()))
m = np.hstack((A, B))

variances = (m_true - m) ** 2

# Make sure things remain 
np.clip(variances, 1e-5, None, out=variances)

# # Kernels
# kernel1 = (GPflow.kernels.Matern32(3, lengthscales=np.sqrt(3)) *
#            GPflow.kernels.Linear(3, variance=variances[0, :], ARD=True))
# kernel2 = (GPflow.kernels.Matern32(3, lengthscales=np.sqrt(3)) *
#            GPflow.kernels.Linear(3, variance=variances[1, :], ARD=True))

kernel1 = (GPflow.kernels.Linear(3, variance=variances[0, :], ARD=True)
           + GPflow.kernels.Matern32(1, lengthscales=1, active_dims=[0])
           * GPflow.kernels.Linear(1, variance=variances[0, 1]))

kernel2 = (GPflow.kernels.Linear(3, variance=variances[1, :], ARD=True)
           + GPflow.kernels.Matern32(1, lengthscales=1, active_dims=[0])
           * GPflow.kernels.Linear(1, variance=variances[1, 1]))

# Mean dynamics

mean_dynamics = safe_learning.LinearSystem((A, B), name='mean_dynamics')
mean_function1 = safe_learning.LinearSystem((A[[0], :], B[[0], :]), name='mean_dynamics_1')
mean_function2 = safe_learning.LinearSystem((A[[1], :], B[[1], :]), name='mean_dynamics_2')

# Define a GP model over the dynamics
gp1 = GPflow.gpr.GPR(np.empty((0, 3), dtype=safe_learning.config.np_dtype),
                    np.empty((0, 1), dtype=safe_learning.config.np_dtype),
                    kernel1,
                    mean_function=mean_function1)
gp1.likelihood.variance = noise_var

gp2 = GPflow.gpr.GPR(np.empty((0, 3), dtype=safe_learning.config.np_dtype),
                    np.empty((0, 1), dtype=safe_learning.config.np_dtype),
                    kernel2,
                    mean_function=mean_function2)
gp2.likelihood.variance = noise_var

gp1_fun = safe_learning.GaussianProcess(gp1)
gp2_fun = safe_learning.GaussianProcess(gp2)

dynamics = safe_learning.FunctionStack((gp1_fun, gp2_fun))


# Get the optimal control gain
a_true, b_true = true_dynamics.linearize()
k_opt, s_opt = safe_learning.utilities.dlqr(a_true, b_true, q, r)
optimal_policy = safe_learning.LinearSystem((-k_opt))
optimal_policy = safe_learning.Saturation(optimal_policy, -1, 1)

In [ ]:

# q = 1 * np.diag([1., 2.])
# r = 1.2 * np.array([[1]], dtype=safe_learning.config.np_dtype)

# # a_true, b_true = true_dynamics.linearize()
# k_opt, s_opt = safe_learning.utilities.dlqr(a_true, b_true, q, r)

# value_function = safe_learning.QuadraticFunction(-s_opt)



# plt.figure(figsize=(10, 10))
# states = policy_disc.all_points

# values = value_function(states).eval()
# values[np.abs(values) >= 50] = 50

# img = plt.imshow(values.reshape(policy_disc.num_points).T,
#                  origin='lower',
#                  extent=rl.value_function.discretization.limits.ravel())
# plt.colorbar(img)

### Reinforcement learning for the mean dynamics

In [ ]:
# Define initial policy
k, s = safe_learning.utilities.dlqr(A, B, q, r)
init_policy = np.clip(policy_disc.all_points.dot(-k.T),
                      action_limits[:, 0],
                      action_limits[:, 1])
policy = safe_learning.Triangulation(policy_disc, init_policy)


gamma = 0.98
value_function = safe_learning.Triangulation(policy_disc, np.zeros_like(init_policy), project=True)


reward_function = safe_learning.QuadraticFunction(linalg.block_diag(-q, -r))


rl = safe_learning.PolicyIteration(
    policy,
    dynamics,
    reward_function,
    value_function,
    gamma=gamma)

with tf.variable_scope('rl_mean_optimization'):
    rl_opt_value_function = rl.optimize_value_function()
#     rl_opt_value_function = rl.value_iteration()

    policy_loss = -tf.reduce_mean(rl.future_values(rl.state_space))

    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
    
    adapt_policy = optimizer.minimize(policy_loss, var_list=[rl.policy.parameters])
    
#     adapt_policy = safe_learning.utilities.gradient_clipping(optimizer,
#                                                              policy_loss,
#                                                              var_list=[rl.policy.parameters],
#                                                              limits=[(-1., 1.)])

    # Clip the policy values to comply with the actuation limits
    adapt_policy = safe_learning.utilities.add_constraint(adapt_policy,
                                                          var_list=[rl.policy.parameters],
                                                          bound_list=action_limits)[0]


In [ ]:
action_space = action_disc.all_points

def rl_optimize_policy():
    rl_opt_value_function.eval(feed_dict=rl.feed_dict)
    for i in range(100):
        adapt_policy.eval(feed_dict=rl.feed_dict)
#     rl.discrete_policy_optimization(action_space)
    

for i in tqdm(range(10)):
    rl_optimize_policy()
# rl_opt_value_function.eval(feed_dict=rl.feed_dict)

In [ ]:
def plot_policy():
    plt.figure(figsize=(10, 10))
    img = plt.imshow(rl.policy.parameters.eval().reshape(policy_disc.num_points).T,
                     origin='lower',
                     extent=rl.policy.discretization.limits.ravel())
    plt.colorbar(img)
    
plot_policy()

In [ ]:
def plot_values():
    plt.figure(figsize=(10, 10))
    img = plt.imshow(rl.value_function.parameters.eval().reshape(policy_disc.num_points).T,
                     origin='lower',
                     extent=rl.value_function.discretization.limits.ravel())
    plt.colorbar(img)
plot_values()

### Define Lyapunov function

In [ ]:
# lyapunov_function = safe_learning.QuadraticFunction(s_opt)
# lipschitz_lyapunov = lambda x: tf.reduce_max(tf.abs(tf.matmul(x, s_opt + s_opt.T)),
#                                              axis=1, keep_dims=True)

lyapunov_function = -rl.value_function
lipschitz_lyapunov = lambda x: tf.reduce_max(tf.abs(rl.value_function.gradient(x)),
                                             axis=1, keep_dims=True)

lipschitz_dynamics = np.max(m_true)

lyapunov = safe_learning.Lyapunov(safety_disc,
                                  lyapunov_function,
                                  dynamics,
                                  lipschitz_dynamics,
                                  lipschitz_lyapunov,
                                  tau,
                                  policy=rl.policy,
                                  initial_set=None)

true_lyapunov = safe_learning.Lyapunov(safety_disc,
                                       lyapunov_function,
                                       true_dynamics,
                                       lipschitz_dynamics,
                                       lipschitz_lyapunov,
                                       tau,
                                       policy=optimal_policy,
                                       initial_set=None)

true_lyapunov_init = safe_learning.Lyapunov(safety_disc,
                                       lyapunov_function,
                                       true_dynamics,
                                       lipschitz_dynamics,
                                       lipschitz_lyapunov,
                                       tau,
                                       policy=policy,
                                       initial_set=None)

# Set initial safe set (level set)
values = safe_learning.QuadraticFunction(s_opt)(safety_disc.all_points).eval()
cutoff = np.max(values) * 0.005


lyapunov.initial_safe_set = np.squeeze(values, axis=1) <= cutoff

true_lyapunov.initial_safe_set = lyapunov.initial_safe_set
true_lyapunov_init.initial_safe_set = lyapunov.initial_safe_set

In [ ]:
def plot_things(lyapunov):
    lyapunov.update_safe_set()
#     lyapunov.safe_set[lyapunov.initial_safe_set] = True
    
    plt.imshow(lyapunov.safe_set.reshape(num_states).T,
               origin='lower',
               extent=lyapunov.discretization.limits.ravel(),
               vmin=0,
               vmax=1)
    
    if isinstance(lyapunov.dynamics, safe_learning.UncertainFunction):
        X = lyapunov.dynamics.functions[0].X
        plt.plot(X[:, 0], X[:, 1], 'rx')
    
    plt.title('safe set')
    plt.colorbar()
    plt.show()
    
# plot_things(lyapunov)
plot_things(lyapunov)
# plot_things(true_lyapunov)
# plot_things(true_lyapunov_init)

## Online learning
As we sample within this initial safe set, we gain more knowledge about the system. In particular, we iteratively select the state withing the safe set, $\mathcal{S}_n$, where the dynamics are the most uncertain (highest variance).

In [ ]:
with tf.variable_scope('opt2'):
    
    # Placeholder for states
    tf_states = tf.placeholder(safe_learning.config.dtype, [None, 2])
    
    # Add Lyapunov uncertainty (but only if safety-relevant)
    values = rl.future_values(tf_states, lyapunov=lyapunov)
    
    policy_loss = -tf.reduce_mean(values)
    

    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
    adapt_policy = optimizer.minimize(policy_loss, var_list=[rl.policy.parameters])
    
    # Clip the policy values to comply with the actuation limits
    adapt_policy = safe_learning.utilities.add_constraint(adapt_policy,
                                                          var_list=[rl.policy.parameters],
                                                          bound_list=action_limits)[0]
    
    
def rl_optimize_policy(num_iter):
    # Optimize value function
    rl_opt_value_function.eval(feed_dict=rl.feed_dict)
    all_states = lyapunov.discretization.all_points

    # select random training batches
    for i in tqdm(range(num_iter)):
        idx = np.random.choice(len(all_states), size=1000, replace=False)
        rl.feed_dict[tf_states] = all_states[idx]

        adapt_policy.eval(feed_dict=rl.feed_dict)

In [ ]:
action_variation = np.array([[-0.02], [0.], [0.02]], dtype=safe_learning.config.np_dtype)


with tf.variable_scope('add_new_measurement'):
        action_dim = lyapunov.policy.output_dim
        tf_max_state_action = tf.placeholder(safe_learning.config.dtype,
                                             shape=[1, safety_disc.ndim + action_dim])
        tf_measurement = true_dynamics(tf_max_state_action)
        
def update_gp():
    """Update the GP model based on an actively selected data point."""
    # Get a new sample location
    max_state_action, _ = safe_learning.get_safe_sample(lyapunov,
                                                        action_variation,
                                                        action_limits,
                                                        num_samples=1000)

    # Obtain a measurement of the true dynamics
    lyapunov.feed_dict[tf_max_state_action] = max_state_action
    measurement = tf_measurement.eval(feed_dict=lyapunov.feed_dict)

    # Add the measurement to our GP dynamics
    lyapunov.dynamics.add_data_point(max_state_action, measurement)
    

In [ ]:
# lyapunov.update_safe_set()
plot_things(lyapunov)

In [ ]:
for i in range(10):
    print(i)
    for i in tqdm(range(10)):
        update_gp()
    
    rl_optimize_policy(num_iter=200)
    lyapunov.update_values()
    lyapunov.update_safe_set()
    
    # Plotting includes an update to the levelset!
    plot_things(lyapunov)
    